In [7]:
import numpy as np 
import pandas as pd 

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
file=pd.read_csv('/content/drive/MyDrive/ML project/anime recommender/anime.csv')
file = file.reset_index()
file.head()

,index,Anime-PlanetID,Name,Alternative Name,Rating Score,Number Votes,Tags,Content Warning,Type,Episodes,Finished,Duration,StartYear,EndYear,Season,Studios,Synopsis,Url
0,0,10,The Prince of Tennis,Tennis no Ouji-sama,4.037,10889,"Comedy, Drama, Shounen, Sports, Tennis, Based ...",Unknown,TV,178,True,Unknown,2001,2005,Fall 2001,"Production I.G, Trans Arts","Meet Ryoma Echizen, the cocky prince of tennis...",https://www.anime-planet.com/anime/the-prince-...
1,1,100,Neon Genesis Evangelion,Shinseiki Evangelion,4.248,54463,"Drama, Mecha, Sci Fi, Conspiracy, Kaijuu, Lone...","Emotional Abuse, Explicit Violence, Mature The...",TV,26,True,Unknown,1995,1996,Fall 1995,"GAINAX, Tatsunoko Production","In the future, a devastating event known as Se...",https://www.anime-planet.com/anime/neon-genesi...
2,2,1000,Full Metal Panic! The Second Raid,Unknown,4.35,23948,"Action, Comedy, Mecha, Sci Fi, Shounen, Milita...","Explicit Violence, Nudity",TV,13,True,Unknown,2005,2005,Summer 2005,Kyoto Animation,Half a year has passed since Sousuke Sagara to...,https://www.anime-planet.com/anime/full-metal-...
3,3,10000,22/7: Shampoo no Nioi ga Shita,Unknown,2.8,131,"Idols, School Life, CG Animation",Unknown,Music,1,True,5,2018,2018,Unknown,Unknown,The idol group 22/7 perform the song Shampoo n...,https://www.anime-planet.com/anime/22-7-shampo...
4,4,10001,Oshiri Tantei: Puputto Fumutto Kaiketsu Dance,Unknown,1.271,21,Family Friendly,Unknown,Web,1,True,2,2017,2017,Unknown,Toei Animation,No synopsis yet - check back soon!,https://www.anime-planet.com/anime/oshiri-tant...


In [11]:
file.columns

Index(['index', 'Anime-PlanetID', 'Name', 'Alternative Name', 'Rating Score',
       'Number Votes', 'Tags', 'Content Warning', 'Type', 'Episodes',
       'Finished', 'Duration', 'StartYear', 'EndYear', 'Season', 'Studios',
       'Synopsis', 'Url'],
      dtype='object')

In [12]:
features=['Rating Score','Number Votes','Studios','Synopsis','Tags','Episodes']

In [13]:
def combined_features(row):
    return str(row['Rating Score'])+" "+ str(row['Number Votes'])+" "+ str(row['Studios'])+" "+ str(row['Synopsis'])+" "+ str(row['Tags'])+" "+ str(row['Episodes'])+" "
def get_title_from_index(index):
    return file[file['index']==index]['Name'].values[0]
def get_index_from_title(title):
    return file[file['Name']==title]['index'].values[0]

In [14]:
file['combined_feature']=file.apply(combined_features,axis=1)
file['combined_feature'].head()

0    4.037 10889 Production I.G, Trans Arts Meet Ry...
1    4.248 54463 GAINAX, Tatsunoko Production In th...
2    4.35 23948 Kyoto Animation Half a year has pas...
3    2.8 131 Unknown The idol group 22/7 perform th...
4    1.271 21 Toei Animation No synopsis yet - chec...
Name: combined_feature, dtype: object

In [15]:
cv=CountVectorizer()
count_matrix=cv.fit_transform(file['combined_feature'])

In [16]:
movie_liked_by_user='Death Note'

In [17]:
cosine_sim=cosine_similarity(count_matrix)

In [18]:
liked_movie_index=cosine_sim[get_index_from_title(movie_liked_by_user)]
similar_anime=list(enumerate(liked_movie_index))
similar_anime.sort(key = lambda row: row[1],reverse=True)
for i in range(10):
    print(get_title_from_index(similar_anime[i][0]))

Death Note
Death Note Rewrite 1: Visions of a God
Rokka: Braves of the Six Flowers
Da Zhu Zai
Sword Art Online: Alicization - War of Underworld: Part II
X TV
Pokemon Movie 2: The Power of One
Sadamitsu the Destroyer
Blast of Tempest
Nagasaki 1945: Angelus no Kane


In [19]:
import pickle
pickle.dump(file.to_dict(),open('./animes_dict.pkl','wb'))
similarity=cosine_sim
pickle.dump(similarity,open('./similarity.pkl','wb'))